In [36]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.11.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [38]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=[
   tfds.Split.TRAIN.subsplit(tfds.percent[:80]),
   tfds.Split.TRAIN.subsplit(tfds.percent[80:90]),
   tfds.Split.TRAIN.subsplit(tfds.percent[90:])
],
    as_supervised=True)

2023-04-02 20:31:06.446758: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 20:31:06.455065: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [39]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by

In [40]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [42]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 1.4732717 ,  0.42393148,  0.16215092,  0.5873862 ,  0.1513372 ,
        -0.5062546 , -0.21033262, -0.33339503, -0.7381036 ,  0.29208365,
        -0.35330555, -0.02719353, -0.25117522,  0.28229505, -0.05980197,
        -0.73671365, -0.55260205,  0.08691197,  0.08244086, -1.3579206 ,
         0.3993887 , -0.5992716 ,  0.73926276,  0.64969933, -0.32441717,
         0.33525264, -1.1349468 ,  0.1502967 ,  0.8454977 , -1.1183991 ,
        -0.8927187 ,  0.9534588 ,  0.22556058, -0.66705996, -0.65366906,
         0.46913755,  0.5412825 , -0.20910901,  0.31495017, -0.91272557,
         0.02957173,  0.1557533 , -0.6307922 ,  0.43090525, -0.46083456,
        -1.018203  , -0.44839182, -0.35512343, -0.41938427,  0.10584122],
       [ 0.42595312,  0.41425827,  0.05380295,  0.65693736,  0.02124309,
        -0.34377313,  0.27501962, -0.29560134, -0.8899341 ,  0.4253666 ,
        -0.03075071,  0.15959139,  0.05135492,  0.41358554, -0.18948269,
 

In [44]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [47]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
40/40 [==============================] - 24s 556ms/step - loss: 0.6167 - accuracy: 0.5869 - val_loss: 0.5229 - val_accuracy: 0.7192
Epoch 2/10
40/40 [==============================] - 23s 562ms/step - loss: 0.4409 - accuracy: 0.7901 - val_loss: 0.3983 - val_accuracy: 0.8068
Epoch 3/10
40/40 [==============================] - 27s 643ms/step - loss: 0.3112 - accuracy: 0.8721 - val_loss: 0.3303 - val_accuracy: 0.8588
Epoch 4/10
40/40 [==============================] - 24s 584ms/step - loss: 0.2257 - accuracy: 0.9143 - val_loss: 0.3053 - val_accuracy: 0.8624
Epoch 5/10
40/40 [==============================] - 27s 648ms/step - loss: 0.1631 - accuracy: 0.9441 - val_loss: 0.2925 - val_accuracy: 0.8772
Epoch 6/10
40/40 [==============================] - 27s 652ms/step - loss: 0.1184 - accuracy: 0.9633 - val_loss: 0.2946 - val_accuracy: 0.8748
Epoch 7/10
40/40 [==============================] - 26s 643ms/step - loss: 0.0850 - accuracy: 0.9771 - val_loss: 0.3037 - val_accuracy: 0.8684

In [48]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

5/5 - 1s - loss: 0.3258 - accuracy: 0.8748 - 963ms/epoch - 193ms/step
loss: 0.326
accuracy: 0.875
